In [1]:
import clipboard, os, shutil, random, json, lorem

In [2]:
import hashlib
def read(file, mode="rb"):
    with open(file, mode) as f:
        return f.read()
def hash(file):
    return hashlib.sha256(read(file)).hexdigest()

In [3]:
folder = 'C:\\Users\\Nathan\\Documents\\Personal\\test\\fvconsole_ddt_sourcefiles'
os.chdir(folder)

for i in range(10):
    name = "source%d.txt"%(i+1)
    with open(name, 'w') as f:
        for i in range(5):
            if i > 0: f.write("\n")
            f.write(lorem.paragraph())

files = os.listdir(".")
for file in files:
    print("%-20s %s"%(file,hash(file)))

In [55]:
def make_expect(filename, checksum):
    with open(filename, 'w') as f:
        f.write(checksum)

def make_calc():
    ddt_calc = 'C:\\Users\\Nathan\\Source\\Repos\\FileVerifier\\FVUnitTest\\data\\case\\calc'
    for i, file in enumerate(os.listdir(),1):
        
        test_case = os.path.join(ddt_calc, "test_basic%d"%i)
        inf = os.path.join(test_case, "in.txt")
        expf = os.path.join(test_case, "expect.txt")
        
        os.makedirs(test_case, exist_ok=True)
        shutil.copy2(file, inf)
        
        checksum = hash(file)
        make_expect(expf, checksum)
#make_calc()

In [56]:
def rand3(files):
    assert len(files) >= 3
    chosen = set()
    while (len(chosen) < 3):
        choice = random.choice(files)
        chosen.add(choice)
    return chosen

def jwrite(obj, file):
    with open(file, 'w') as f:
        json.dump(obj, f, indent=4)

        
def delfiles(folder):
    try:
        for file in os.listdir(folder):
            fullpath = os.path.join(folder, file)
            try:
                os.remove(fullpath)
            except (FileNotFoundError, PermissionError):
                pass
    except FileNotFoundError:
        pass
        
def test_folder(root, name):
    folder = os.path.join(root, name)
    delfiles(folder)
    os.makedirs(folder, exist_ok=True)
    return folder
        
def make_data(test_case, chosen):
    data = []
    for file in chosen:
        name, ext = os.path.splitext(file)
        fdata = {
            "source": file,
            "target": name+"_target"+ext
        }
        data.append(fdata)
    return data


def make_expdata(data):
    return {fdata["target"]:hash(fdata["source"]) for fdata in data}
    
def make_create():
    ddt_create = 'C:\\Users\\Nathan\\Source\\Repos\\FileVerifier\\FVUnitTest\\data\\case\\create'
    files = os.listdir()
    for i in range(1,4):

        chosen = rand3(files)
        
        # batch mode - no append
        test_case = test_folder(ddt_create, "test_batch_basic%d"%i)
        inf = os.path.join(test_case, "in.json")
        expf = os.path.join(test_case, "expect.json")
        outf = os.path.join(test_case, "out.json")
        
        data = make_data(test_case, chosen)
        expdata = make_expdata(data)
        
        jwrite(data, inf)
        jwrite(expdata, expf)
        
        for fdata in data:
            source = fdata["source"]
            dst = os.path.join(test_case, source)
            shutil.copy2(source, dst)
            
        # batch mode - append
        test_case = test_folder(ddt_create, "test_batch_append%d"%i)
        inf = os.path.join(test_case, "in.json")
        expf = os.path.join(test_case, "expect.json")
        outf = os.path.join(test_case, "out.json")
        
        predata = {"foo": "bar", "random_filename": "random_hash"}
        
        data = make_data(test_case, chosen)
        expdata = make_expdata(data)
        expdata.update(predata)
        
        jwrite(data, inf)
        jwrite(expdata, expf)
        jwrite(predata, outf)
        
        for fdata in data:
            source = fdata["source"]
            dst = os.path.join(test_case, source)
            shutil.copy2(source, dst)
        
        # single mode - append
        test_case = test_folder(ddt_create, "test_single_append%d"%i)
        inf = os.path.join(test_case, "in.json")
        expf = os.path.join(test_case, "expect.json")
        outf = os.path.join(test_case, "out.json")
        tgtf = os.path.join(test_case, "target.txt")
        
        data = read(random.choice(files), 'r')
        with open(tgtf, 'w') as f:
            f.write("The contents of this file are unused - presence of file is used to indicate ")
        with open(inf, 'w') as f:
            f.write(data)
        jwrite(predata, outf)
        newexpdata = predata.copy()
        newexpdata["target.txt"] = hash(inf)
        jwrite(newexpdata, expf)
        print("Success")
#make_create()      
        

Success
Success
Success


In [57]:
def make_verify():
    ddt_verify = 'C:\\Users\\Nathan\\Source\\Repos\\FileVerifier\\FVUnitTest\\data\\case\\verify'
    # batch mode
    for i in range(3):
        files = os.listdir()
        chosen = rand3(files)
        
        data = {}
        for file in chosen:
            data[file] = hash(file)
            
        test_case = test_folder(ddt_verify, "test_single%d"%(i+1))
        inf = os.path.join(test_case, "in.json")
        jwrite(data, inf)
        for source in chosen:
            dst = os.path.join(test_case, source)
            shutil.copy2(source, dst)
        print("Success")
#make_verify()
            

Success
Success
Success


In [74]:
def make_calc_ftp():
    ddt_calc = 'C:\\Users\\Nathan\\Source\\Repos\\FileVerifier\\FVUnitTest\\data\\case\\calc'
    for i, file in enumerate(os.listdir(),1):
        if i > 2: break
        test_case = os.path.join(ddt_calc, "test_ftp%d"%i)
        inf = os.path.join(test_case, "in.txt")
        expf = os.path.join(test_case, "expect.txt")
        ftpf = os.path.join(test_case, "ftp.txt")
        
        os.makedirs(test_case, exist_ok=True)
        shutil.copy2(file, inf)
        with open(ftpf, 'w') as f:
            f.write("ftp://%s:%s@%s/"%("fvconsole_user", "uRttUR3MRsF4z6UUngZ3", "127.0.0.1"))
        checksum = hash(file)
        make_expect(expf, checksum)
        
make_calc_ftp()

In [66]:
from pyftpdlib.authorizers import DummyAuthorizer
from pyftpdlib.handlers import FTPHandler
from pyftpdlib.servers import FTPServer

import threading
class ServerThread(threading.Thread):
    
    @classmethod
    def RunSingleTonOrWHatever(cls):
        authorizer = DummyAuthorizer()
#         authorizer.add_user("user", "12345", ".", perm="elradfmwMT")
        authorizer.add_anonymous(".")
        authorizer.add_anonymous("./foofolder")

        handler = FTPHandler
        handler.authorizer = authorizer
        if hasattr(cls, "server"):
            cls.server.close_all()
        cls.server = FTPServer(("127.0.0.1", 21), handler)
        cls.server.serve_forever()
    def run(self):
        self.RunSingleTonOrWHatever()

    def stop(self):
        self.__class__.server.close_all()
        del self.__class__.server
        self.join()
    def __del__(self, *args):
        try:
            self.stop()
        except:
            pass


In [67]:
from ftplib import FTP
thread = ServerThread()
thread.start()
ftp = FTP("localhost")
ftp.login()
ftp.retrlines('LIST')
ftp.quit()
thread.stop()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\program files\python35\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "<ipython-input-66-856960aa7c71>", line 22, in run
    self.RunSingleTonOrWHatever()
  File "<ipython-input-66-856960aa7c71>", line 13, in RunSingleTonOrWHatever
    authorizer.add_anonymous("./foofolder")
  File "c:\program files\python35\lib\site-packages\pyftpdlib\authorizers.py", line 135, in add_anonymous
    DummyAuthorizer.add_user(self, 'anonymous', '', homedir, **kwargs)
  File "c:\program files\python35\lib\site-packages\pyftpdlib\authorizers.py", line 103, in add_user
    raise ValueError('user %r already exists' % username)
ValueError: user 'anonymous' already exists



ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [79]:
ftp = FTP("localhost")
ftp.login("fvconsole_user", "uRttUR3MRsF4z6UUngZ3")
ftp.retrlines('LIST')
ftp.quit()

-rw-rw-rw-   1 owner    group        1106 Mar 06 01:37 ftp_host.py
-rw-rw-rw-   1 owner    group        1601 Mar 05 00:38 launch_console.py
-rw-rw-rw-   1 owner    group        2988 Mar 06 00:30 remote_console.py


'221 Goodbye.'